### Set up pipeline with basic imports, data directories, and file basenames.

In [ ]:
import os
from benbiohelpers.DataPipelineManagement.GenomeManager import getGenomeFastaFilePath
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import getExternalDataDirectory as getMutperiodExternalDataDirectory

hg19FastaFilePath = getGenomeFastaFilePath("hg19")
mutperiodHg19Directory = getMutperiodExternalDataDirectory()

liDataDirectory = os.path.join(getDataDir(), "Li_tXR-seq")
liBaseDataFilePath = os.path.join(liDataDirectory, "Li_tXR-seq.bed")

jiang2uM_CellularDataDir = os.path.join(getDataDir(), "Jiang_BPDE_damage_maps", "BEAS-2B_2uM_BPDE_cell_24h")

### Parse the tXR-seq data for mutperiod input

In [ ]:
from mutperiodpy.input_parsing.ParseXRSeq import parseXRSeq, PresetCallParams
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory

if getFilesInDirectory(liDataDirectory, DataTypeStr.mutations + ".bed", searchRecursively = False) is None:
    print("Parsed data not found. Generating...")
    liMutperiodInputFilePaths = parseXRSeq([liBaseDataFilePath], PresetCallParams.BPDE, hg19FastaFilePath)
else:
    liMutperiodInputFilePaths = [getFilesInDirectory(liDataDirectory, DataTypeStr.mutations + ".bed", searchRecursively = False)]
    print(f"Found parsed data at {liMutperiodInputFilePaths[0]}. Continuing.")

### Run the mutperiod pipeline to count repair events relative to transcription factor binding sites.
NOTE: This analysis requires that the damage data has already been parsed for mutperiod using the Jiang_damage_maps_nucleosome_periodicity_data_generation notebook.

In [ ]:
from mutperiodpy.RunAnalysisSuite import runAnalysisSuite
TSS_FilePaths = list()
TSS_FilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_protein_coding_genes_TSSs", "hg19_protein_coding_genes_TSSs.bed"))

TSS_Names = [os.path.basename(TSS_FilePath).rsplit('.', 1)[0] for TSS_FilePath in TSS_FilePaths]

runAnalysisSuite(liMutperiodInputFilePaths, TSS_Names,
                 normalizationMethod = "No Normalization", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True, useNucStrand = True)
runAnalysisSuite(liMutperiodInputFilePaths, TSS_Names,
                 normalizationMethod = "Singlenuc/Dinuc", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True, useNucStrand = True)
runAnalysisSuite(liMutperiodInputFilePaths, TSS_Names,
                 normalizationMethod = "Custom Background", customBackgroundDir = jiang2uM_CellularDataDir,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True, useNucStrand = True)
runAnalysisSuite(liMutperiodInputFilePaths, TSS_Names,
                 normalizationMethod = "Trinuc/Quadrunuc", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True, useNucStrand = True)